## Audio Feature Extraction

**Packages**

In [1]:
import h5py
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
from tqdm import tqdm
import os.path as osp
import torch
import numpy as np
from moviepy.video.io.VideoFileClip import VideoFileClip

### Loading VGGish model

In [2]:
import torch
vggish = torch.hub.load('harritaylor/torchvggish', 'vggish')
vggish.eval()


Using cache found in C:\Users\Msc 2/.cache\torch\hub\harritaylor_torchvggish_master


VGGish(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False

### Feature Extracter

In [3]:
def get_segFeats(path2vid, picks):
    audio = VideoFileClip(path2vid).audio
    if audio == None:
        Audio_feats=np.zeros((len(picks),128))
        return Audio_feats
        
    dur=audio.duration
    Audio_Feats=[]
    
    pbar = tqdm(total=len(picks), position=0, leave=True)
    
    first_sarr = audio.cutout(0.5,dur).to_soundarray(fps=16000)
    pad_amt = 16000-first_sarr.shape[0]
    first_sarr= np.pad(first_sarr, ((pad_amt,0),(0,0)), 'constant')
    feat = vggish.forward(first_sarr, fs=16000).detach().numpy()
    Audio_Feats.append(feat)
    pbar.update(1)
    
    i=0.0   ##
    for pick in picks[1:]:
        # i=pick/30           ### pick/fps
        i+=0.5
        audio_clip = audio.cutout(0,i-0.5).cutout(min(i+0.5,dur),dur)
        audio_arr = audio_clip.to_soundarray(fps=16000)
        pad_amt = 16000-audio_arr.shape[0]
        if pad_amt>0:
            audio_arr= np.pad(audio_arr, ((0, pad_amt),(0,0)), 'constant')
        
        feat = vggish.forward(audio_arr, fs=16000).detach().numpy()
        Audio_Feats.append(feat)
        pbar.update(1)
        
    return Audio_Feats
        

In [4]:
def extract_audio_feats(args):
    with h5py.File(args['output_h5'], 'a') as d:
        for key in d.keys():
            if 'aud_feats' in d[key].keys():
                continue
            print("Extracting Audio Feature from : ", key)
            video_name=d[key+'/video_name'][()].decode()
            path2vid = args['video_folder']+'/'+video_name
            picks = d[key+'/picks'][()]
            
            audio_feats = np.array(get_segFeats(path2vid, picks))

            print('Feature Shape: ', audio_feats.shape)
            
            d.create_dataset(key + '/aud_feats', data=audio_feats)
            
            

### TVSum Audio Feature Extraction

In [22]:
# args={
#     'output_h5':'extracted_features/normal/TVSum05s.h5',
#     'video_folder':'../data/Public datasets/ydata-tvsum50-v1_1/video'
# }

In [24]:
# extract_audio_feats(args)

Extracting Audio Feature from :  video_1


100%|██████████| 708/708 [00:29<00:00, 23.60it/s]


Feature Shape:  (708, 128)
Extracting Audio Feature from :  video_10


100%|██████████| 267/267 [00:11<00:00, 24.06it/s]


Feature Shape:  (267, 128)
Extracting Audio Feature from :  video_11


...

Feature Shape:  (298, 128)
Extracting Audio Feature from :  video_8


100%|██████████| 659/659 [00:27<00:00, 23.97it/s]


Feature Shape:  (659, 128)
Extracting Audio Feature from :  video_9


100%|██████████| 468/468 [00:18<00:00, 25.59it/s]

Feature Shape:  (468, 128)


### Summe Audio Feature Extraction

In [18]:
# args={
#     'output_h5':'extracted_features/normal/SUMMe.h5',
#     'video_folder':'../data/Public datasets/SUMMe/videos'
# }

In [21]:
# extract_audio_feats(args)

Extracting Audio Feature from :  video_1


100%|██████████| 299/299 [00:10<00:00, 28.17it/s]


Feature Shape:  (299, 128)
Extracting Audio Feature from :  video_10


100%|██████████| 331/331 [00:12<00:00, 27.58it/s]


Feature Shape:  (331, 128)
Extracting Audio Feature from :  video_11


...

Feature Shape:  (338, 128)
Extracting Audio Feature from :  video_8


100%|██████████| 603/603 [00:22<00:00, 27.35it/s]


Feature Shape:  (603, 128)
Extracting Audio Feature from :  video_9


100%|██████████| 85/85 [00:02<00:00, 28.65it/s]

Feature Shape:  (85, 128)


--------------

In [5]:
args={
    'output_h5':'extracted_features/normal/customSet.h5',
    'video_folder':'../data/Custom dataset'
}
extract_audio_feats(args)

Extracting Audio Feature from :  video_1


100%|██████████| 235/235 [00:08<00:00, 26.30it/s]


Feature Shape:  (235, 128)
Extracting Audio Feature from :  video_2


100%|██████████| 323/323 [00:12<00:00, 25.97it/s]


Feature Shape:  (323, 128)
Extracting Audio Feature from :  video_3


...

Feature Shape:  (9749, 128)
Extracting Audio Feature from :  video_4


100%|██████████| 884/884 [00:37<00:00, 23.67it/s]


Feature Shape:  (884, 128)
Extracting Audio Feature from :  video_5


100%|██████████| 354/354 [00:14<00:00, 25.25it/s]

Feature Shape:  (354, 128)


In [17]:
import torch
torch.__version__

'2.0.0'

In [3]:
# audio = VideoFileClip('../data/Public datasets/ydata-tvsum50-v1_1/video/sTEELN-vY30.mp4').audio


# dur=audio.duration


# first_sarr = audio.cutout(0.5,dur).to_soundarray(fps=16000)